# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 18 2022 11:47AM,237546,19,GP05182022-3,"Granules Pharmaceuticals, Inc.",Released
1,May 18 2022 11:41AM,237545,10,8035807,Eywa Pharma Inc.,Released
2,May 18 2022 11:41AM,237545,10,8035823,Eywa Pharma Inc.,Released
3,May 18 2022 11:41AM,237545,10,8035825,Eywa Pharma Inc.,Released
4,May 18 2022 11:41AM,237545,10,8035826,Eywa Pharma Inc.,Released
5,May 18 2022 11:41AM,237545,10,8035833,Eywa Pharma Inc.,Released
6,May 18 2022 11:41AM,237545,10,8035834,Eywa Pharma Inc.,Released
7,May 18 2022 11:41AM,237545,10,8035836,Eywa Pharma Inc.,Released
8,May 18 2022 11:41AM,237545,10,8035837,Eywa Pharma Inc.,Released
9,May 18 2022 11:41AM,237545,10,8035840,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
86,237541,Released,1
87,237543,Released,18
88,237544,Released,25
89,237545,Released,21
90,237546,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
237541,NaN,NaN,1.0
237543,NaN,NaN,18.0
237544,NaN,NaN,25.0
237545,NaN,NaN,21.0
237546,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237404,0.0,10.0,3.0
237409,4.0,1.0,0.0
237415,0.0,0.0,1.0
237425,0.0,0.0,1.0
237427,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237404,0,10,3
237409,4,1,0
237415,0,0,1
237425,0,0,1
237427,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237404,0,10,3
1,237409,4,1,0
2,237415,0,0,1
3,237425,0,0,1
4,237427,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237404,,10,3
1,237409,4,1,
2,237415,,,1
3,237425,,,1
4,237427,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 18 2022 11:47AM,237546,19,"Granules Pharmaceuticals, Inc."
1,May 18 2022 11:41AM,237545,10,Eywa Pharma Inc.
22,May 18 2022 11:40AM,237544,10,Eywa Pharma Inc.
47,May 18 2022 11:34AM,237543,10,Beach Products Inc
65,May 18 2022 11:20AM,237541,12,"SD Head USA, LLC"
66,May 18 2022 11:18AM,237540,16,Sartorius Lab Products and Service
79,May 18 2022 11:17AM,237539,16,Sartorius Bioprocess Solutions
89,May 18 2022 11:17AM,237538,16,Sartorius Bioprocess Solutions
90,May 18 2022 11:06AM,237536,18,"Blinc International, Inc.-Blinc-ecom"
91,May 18 2022 11:04AM,237535,19,ACG North America LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 18 2022 11:47AM,237546,19,"Granules Pharmaceuticals, Inc.",,,1
1,May 18 2022 11:41AM,237545,10,Eywa Pharma Inc.,,,21
2,May 18 2022 11:40AM,237544,10,Eywa Pharma Inc.,,,25
3,May 18 2022 11:34AM,237543,10,Beach Products Inc,,,18
4,May 18 2022 11:20AM,237541,12,"SD Head USA, LLC",,,1
5,May 18 2022 11:18AM,237540,16,Sartorius Lab Products and Service,,1,12
6,May 18 2022 11:17AM,237539,16,Sartorius Bioprocess Solutions,,1,9
7,May 18 2022 11:17AM,237538,16,Sartorius Bioprocess Solutions,,,1
8,May 18 2022 11:06AM,237536,18,"Blinc International, Inc.-Blinc-ecom",,,1
9,May 18 2022 11:04AM,237535,19,ACG North America LLC,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 18 2022 11:47AM,237546,19,"Granules Pharmaceuticals, Inc.",1,,
1,May 18 2022 11:41AM,237545,10,Eywa Pharma Inc.,21,,
2,May 18 2022 11:40AM,237544,10,Eywa Pharma Inc.,25,,
3,May 18 2022 11:34AM,237543,10,Beach Products Inc,18,,
4,May 18 2022 11:20AM,237541,12,"SD Head USA, LLC",1,,
5,May 18 2022 11:18AM,237540,16,Sartorius Lab Products and Service,12,1,
6,May 18 2022 11:17AM,237539,16,Sartorius Bioprocess Solutions,9,1,
7,May 18 2022 11:17AM,237538,16,Sartorius Bioprocess Solutions,1,,
8,May 18 2022 11:06AM,237536,18,"Blinc International, Inc.-Blinc-ecom",1,,
9,May 18 2022 11:04AM,237535,19,ACG North America LLC,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 18 2022 11:47AM,237546,19,"Granules Pharmaceuticals, Inc.",1,,
1,May 18 2022 11:41AM,237545,10,Eywa Pharma Inc.,21,,
2,May 18 2022 11:40AM,237544,10,Eywa Pharma Inc.,25,,
3,May 18 2022 11:34AM,237543,10,Beach Products Inc,18,,
4,May 18 2022 11:20AM,237541,12,"SD Head USA, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 18 2022 11:47AM,237546,19,"Granules Pharmaceuticals, Inc.",1.0,NaN,NaN
1,May 18 2022 11:41AM,237545,10,Eywa Pharma Inc.,21.0,NaN,NaN
2,May 18 2022 11:40AM,237544,10,Eywa Pharma Inc.,25.0,NaN,NaN
3,May 18 2022 11:34AM,237543,10,Beach Products Inc,18.0,NaN,NaN
4,May 18 2022 11:20AM,237541,12,"SD Head USA, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 18 2022 11:47AM,237546,19,"Granules Pharmaceuticals, Inc.",1.0,0.0,0.0
1,May 18 2022 11:41AM,237545,10,Eywa Pharma Inc.,21.0,0.0,0.0
2,May 18 2022 11:40AM,237544,10,Eywa Pharma Inc.,25.0,0.0,0.0
3,May 18 2022 11:34AM,237543,10,Beach Products Inc,18.0,0.0,0.0
4,May 18 2022 11:20AM,237541,12,"SD Head USA, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4274870,149.0,53.0,38.0
12,237541,1.0,0.0,0.0
15,1662488,32.0,11.0,0.0
16,950091,22.0,3.0,0.0
18,6411667,26.0,16.0,0.0
19,1424884,16.0,13.0,6.0
20,1899651,14.0,28.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4274870,149.0,53.0,38.0
1,12,237541,1.0,0.0,0.0
2,15,1662488,32.0,11.0,0.0
3,16,950091,22.0,3.0,0.0
4,18,6411667,26.0,16.0,0.0
5,19,1424884,16.0,13.0,6.0
6,20,1899651,14.0,28.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,149.0,53.0,38.0
1,12,1.0,0.0,0.0
2,15,32.0,11.0,0.0
3,16,22.0,3.0,0.0
4,18,26.0,16.0,0.0
5,19,16.0,13.0,6.0
6,20,14.0,28.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,149.0
1,12,Released,1.0
2,15,Released,32.0
3,16,Released,22.0
4,18,Released,26.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20
Status,,,,,,,
Completed,38.0,0.0,0.0,0.0,0.0,6.0,0.0
Executing,53.0,0.0,11.0,3.0,16.0,13.0,28.0
Released,149.0,1.0,32.0,22.0,26.0,16.0,14.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20
0,Completed,38.0,0.0,0.0,0.0,0.0,6.0,0.0
1,Executing,53.0,0.0,11.0,3.0,16.0,13.0,28.0
2,Released,149.0,1.0,32.0,22.0,26.0,16.0,14.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20
0,Completed,38.0,0.0,0.0,0.0,0.0,6.0,0.0
1,Executing,53.0,0.0,11.0,3.0,16.0,13.0,28.0
2,Released,149.0,1.0,32.0,22.0,26.0,16.0,14.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()